# plotting healpix data with xdggs and lonboard

We will show in this notebook, several example of plotting healpix data.  
This notebook is made for a poster presentation of 3rd Destination Earth User eXchange
October 15 - October 16 2024 held in darmstadt rese 3rd Destination EArth https://destination-earth.eu/event/3rd-destination-earth-user-exchange/ 

Link to the Poster in zenodo here



For this to work, we need to install xdggs exist in lonboard branch at https://github.com/keewis/xdggs.git@lonboard
Please use `explore` extra using following command
`pip install "xdggs[explore] @ git+https://github.com/keewis/xdggs.git@lonboard"`

It will install all requried libraries such as 
- `xarray`
- `numpy`
- `xdggs`
- `lonboard`
- `arro3-core`
- `matplotlib`

## Why in xdggs we use lonboard for plotting ?
Lonboard use GPU-based rendering, specifically leveraging WebGL through deck.gl, because it allows the efficient handling of large datasets. By rendering on the GPU, it enables the fast, interactive visualization of geospatial data directly in a browser, which is crucial when dealing with large-scale datasets that would otherwise slow down performance or crash typical plotting libraries.

The core of Lonboard's performance comes from its use of binary data formats like GeoArrow and GeoParquet, which minimize data transfer and processing overhead. Unlike traditional libraries that use text formats such as GeoJSON, Lonboard operates in a binary format throughout the entire pipeline. This binary handling reduces the number of copies made and avoids costly conversions between formats, resulting in faster data transfers and more efficient rendering





In [1]:
#Lets install our latest xdggs package with plotting functionality
!pip install "xdggs[explore] @ git+https://github.com/keewis/xdggs.git@lonboard"

  Cloning https://github.com/keewis/xdggs.git (to revision lonboard) to /tmp/pip-install-qf0bkvzn/xdggs_1dc7d439d2d547c58ee1b55be858c4d9
  Running command git clone --filter=blob:none --quiet https://github.com/keewis/xdggs.git /tmp/pip-install-qf0bkvzn/xdggs_1dc7d439d2d547c58ee1b55be858c4d9
  Running command git checkout -b lonboard --track origin/lonboard
  Switched to a new branch 'lonboard'
  Branch 'lonboard' set up to track remote branch 'lonboard' from 'origin'.
  Resolved https://github.com/keewis/xdggs.git to commit 6d9b78aeb68b03a2a196e4a14b99a73b8dc6538c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import numpy as np
import xarray as xr
import xdggs

##  Plotting generated test dataset

Following code with generate a test dateset. 

In [3]:
level = 8
cell_ids = np.arange(12 * 4**level)
grid_info = {"grid_name": "healpix", "level": level, "indexing_scheme": "nested"}

ds = (
    xr.Dataset(coords={"cell_ids": ("cells", cell_ids, grid_info)})
    .pipe(xdggs.decode)
    .pipe(lambda ds: ds.merge(ds.dggs.cell_centers()))
    .assign(data=lambda ds: np.cos(6 * np.radians(ds["latitude"])) * np.sin(6 * np.radians(ds["longitude"])))
)
ds

<xarray.Dataset> Size: 25MB
Dimensions:    (cells: 786432)
Coordinates:
  * cell_ids   (cells) int64 6MB 0 1 2 3 4 ... 786428 786429 786430 786431
    latitude   (cells) float64 6MB 0.1492 0.2984 0.2984 ... -0.2984 -0.1492
    longitude  (cells) float64 6MB 45.0 45.18 44.82 45.0 ... 315.2 314.8 315.0
Dimensions without coordinates: cells
Data variables:
    data       (cells) float64 6MB -0.9999 -0.9993 -0.9993 ... 0.9993 0.9999
Indexes:
    cell_ids  HealpixIndex(nside=8, indexing_scheme=nested, rotation=[0.0, 0.0])

### Plotting with xdggs is very simple, just use 'explore' function

In [4]:
ds["data"].dggs.explore(center=0, cmap="coolwarm",alpha=1)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f3ca1ba…

## Open a sample DestinE data in hearlpix (saved in zarr format with attributes)

It is straigh forward to open dataset in xarray to use an annotated zarr file than using the grib file accessible from DEDL.  

Here we have loaded grib file then transformed to zarr file with following metadata, and assigned coordinate.  

```.assign_coords(
    cell_ids=('value', np.arange(12*4**10 ), 
              {"grid_name":"healpix","level": 10, "indexing_scheme": "nested"}))
```

In [5]:
ds = xr.open_dataset(
    "https://data-taos.ifremer.fr/DestinE/average_surface_temperature.zarr",
    engine="zarr",
    chunks={},
    consolidated=True,
)
ds

<xarray.Dataset> Size: 4GB
Dimensions:          (oceanModelLayer: 75, cells: 12582912)
Coordinates:
    cell_ids         (cells) int64 101MB dask.array<chunksize=(100000,), meta=np.ndarray>
    latitude         (cells) float64 101MB dask.array<chunksize=(100000,), meta=np.ndarray>
    longitude        (cells) float64 101MB dask.array<chunksize=(100000,), meta=np.ndarray>
  * oceanModelLayer  (oceanModelLayer) float64 600B 1.0 2.0 3.0 ... 74.0 75.0
    step             timedelta64[ns] 8B ...
    time             datetime64[ns] 8B ...
    valid_time       datetime64[ns] 8B ...
Dimensions without coordinates: cells
Data variables:
    avg_thetao       (oceanModelLayer, cells) float32 4GB dask.array<chunksize=(10, 100000), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            2
    GRIB_subCentre:          1003
    history:                 2024-06-03T13:52 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [6]:
data=(
    ds["avg_thetao"]
    .isel(oceanModelLayer=1)
    .compute()
    .pipe(xdggs.decode)
)
data

<xarray.DataArray 'avg_thetao' (cells: 12582912)> Size: 50MB
array([300.53537, 300.54025, 300.50998, ..., 301.4836 , 301.5012 ,
       301.49338], dtype=float32)
Coordinates:
  * cell_ids         (cells) int64 101MB 0 1 2 3 ... 12582909 12582910 12582911
    latitude         (cells) float64 101MB 0.0373 0.0746 ... -0.0746 -0.0373
    longitude        (cells) float64 101MB 45.0 45.04 44.96 ... 315.0 315.0
    oceanModelLayer  float64 8B 2.0
    step             timedelta64[ns] 8B 1 days
    time             datetime64[ns] 8B 2023-01-01
    valid_time       datetime64[ns] 8B 2023-01-02
Dimensions without coordinates: cells
Indexes:
    cell_ids  HealpixIndex(nside=10, indexing_scheme=nested, rotation=[0.0, 0.0])
Attributes: (12/18)
    GRIB_NV:                         0
    GRIB_cfName:                     unknown
    GRIB_cfVarName:                  avg_thetao
    GRIB_dataType:                   fc
    GRIB_gridDefinitionDescription:  150
    GRIB_gridType:                   healpix
    ...                              ...
    GRIB_stepUnits:                  1
    GRIB_typeOfLevel:                oceanModelLayer
    GRIB_units:                      K
    long_name:                       Time-mean sea water potential temperature
    standard_name:                   unknown
    units:                           K

In [7]:
data.dggs.explore(center=0, 
                  cmap="viridis"
    ,alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f3c8688…

In [8]:
import pint_xarray
import cf_xarray.units
data = (
    data
    .pint.quantify()
    .pint.to({"avg_thetao": "degC"})
    .pint.dequantify()
)
data


<xarray.DataArray 'avg_thetao' (cells: 12582912)> Size: 50MB
array([27.385376, 27.390259, 27.359985, ..., 28.333618, 28.351196,
       28.343384], dtype=float32)
Coordinates:
    cell_ids         (cells) int64 101MB 0 1 2 3 ... 12582909 12582910 12582911
    latitude         (cells) float64 101MB 0.0373 0.0746 ... -0.0746 -0.0373
    longitude        (cells) float64 101MB 45.0 45.04 44.96 ... 315.0 315.0
    oceanModelLayer  float64 8B 2.0
    step             timedelta64[ns] 8B 1 days
    time             datetime64[ns] 8B 2023-01-01
    valid_time       datetime64[ns] 8B 2023-01-02
Dimensions without coordinates: cells
Attributes: (12/18)
    GRIB_NV:                         0
    GRIB_cfName:                     unknown
    GRIB_cfVarName:                  avg_thetao
    GRIB_dataType:                   fc
    GRIB_gridDefinitionDescription:  150
    GRIB_gridType:                   healpix
    ...                              ...
    GRIB_stepUnits:                  1
    GRIB_typeOfLevel:                oceanModelLayer
    GRIB_units:                      K
    long_name:                       Time-mean sea water potential temperature
    standard_name:                   unknown
    units:                           degree_Celsius

In [9]:
data.pipe(xdggs.decode).dggs.explore(center=0, 
                  cmap="viridis"
    ,alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f3c868a…

## Open a EERIE Cloud  data in hearlpix, saved in zarr format from DKRZ.

NextGems project provide their data set in healpix in zarr format. 
Deatiled description of EERIE Cloud can be found here
https://easy.gems.dkrz.de/simulations/EERIE/eerie_data-access_online.html

After opening we need to assigne coordinate and meta data defining the grid system.    



In [10]:

ds = xr.open_dataset(
    "https://eerie.cloud.dkrz.de/datasets/nextgems.IFS_9-FESOM_5-production.2D_hourly_healpix512/zarr",
    engine="zarr",
    chunks={},
    consolidated=True,
)
ds

<xarray.Dataset> Size: 383TB
Dimensions:  (time: 262537, value: 3145728)
Coordinates:
    lat      (value) float64 25MB dask.array<chunksize=(3145728,), meta=np.ndarray>
    lon      (value) float64 25MB dask.array<chunksize=(3145728,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2MB 2020-01-20 ... 2050-01-01
Dimensions without coordinates: value
Data variables: (12/58)
    10si     (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    10u      (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    10v      (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    2d       (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    2t       (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    blh      (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    ...       ...
    tp       (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    tprate   (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    tsr      (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    tsrc     (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    ttr      (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    ttrc     (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
Attributes: (12/13)
    edition:            2
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          1003
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...
    title:              nextgems.IFS_9-FESOM_5-production.2D_hourly_healpix512
    ...                 ...
    time_min:           2020-01-20T00:00:00.000000000
    time_max:           2050-01-01T00:00:00.000000000
    creation_date:      2024-10-13T00:02:17Z
    authors:            Wieners, Karl-Hermann (MPIMet), Aguridan, Razvan (ECM...
    contact:            Wieners, Karl-Hermann (MPIMet), Rackow, Thomas (ECMWF)
    license:            Creative Commons Attribution 4.0 International (CC BY...

In [11]:
ds=(ds
    .assign_coords(
        cell_ids=('value', np.arange(12*4**9 ), 
                  {"grid_name":"healpix","level": 9, "indexing_scheme": "nested"})
    )
   )

In [12]:
data=ds["msl"].isel(time=1000).compute().pipe(xdggs.decode)
data

<xarray.DataArray 'msl' (value: 3145728)> Size: 25MB
array([100984.51660156, 100988.79394531, 100979.44824219, ...,
       101204.87207031, 101202.56152344, 101200.22753906])
Coordinates:
    lat       (value) float64 25MB 0.0746 0.1492 0.1492 ... -0.1492 -0.0746
    lon       (value) float64 25MB 45.0 45.09 44.91 45.0 ... 315.1 314.9 315.0
    time      datetime64[ns] 8B 2020-03-01T16:00:00
  * cell_ids  (value) int64 25MB 0 1 2 3 4 ... 3145724 3145725 3145726 3145727
Dimensions without coordinates: value
Indexes:
    cell_ids  HealpixIndex(nside=9, indexing_scheme=nested, rotation=[0.0, 0.0])
Attributes: (12/15)
    paramId:                    151
    dataType:                   fc
    numberOfPoints:             3145728
    typeOfLevel:                meanSea
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    name:                       Mean sea level pressure
    cfName:                     air_pressure_at_mean_sea_level
    cfVarName:                  msl
    missingValue:               9999
    NV:                         0
    gridDefinitionDescription:  150

In [13]:
data.dggs.explore(center=0, 
                  cmap="viridis"
    ,alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f3c12c1…

### Selecting region to plot
#### Selection using healpix cell_ids

In [14]:

data.sel(cell_ids=slice(0,1000)).dggs.explore(#center=0, 
                  cmap="viridis"
    ,alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f3c0ef6…

#### Selection using latitude and longitude

In [15]:
(data
 .dggs
 .sel_latlon(latitude=50, longitude=10)
 .expand_dims("cell_ids")
 .pipe(xdggs.decode)
).dggs.explore(center=0, 
                  cmap="viridis"
    ,alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f3c0ef6…

In [16]:
(data
 .dggs
 .sel_latlon(longitude=[48,47,46], latitude=[-5,-5.3,-5.5])
).dggs.explore(center=0,                 cmap="viridis",alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f3c0ef6…

In [17]:
data_subset=data.dggs.assign_latlon_coords()
data_subset=data_subset.where(
    (data_subset.longitude >= 46) & (data_subset.longitude <= 48) &
    (data_subset.latitude >= -5) & (data_subset.latitude <= 0), 
    drop=True
)
data_subset.dggs.explore(center=0,                 cmap="viridis",alpha=0.8)

#.where( latitude

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f3c0ef6…

## Current Issue with Testing Different Datasets

It is crucial to define a consistent coordinate 'name' and establish how attributes should be specified in order to test different data sources without having to re-define the coordinate name for each dataset. In **XDGGS**, we follow the convention below:

```python
.assign_coords(
    cell_ids=('value', np.arange(12*4**10), 
              {"grid_name": "healpix", "level": 10, "indexing_scheme": "nested"}))
```

## Whats next

Currently, selecting data by latitude and longitude must be done on a point-by-point basis. However, the conventional approach for users to select a region of interest is still a bounding box (bbox) using latitude and longitude. Therefore, enabling this functionality is a priority to streamline data selection.